## Tracking
このコードでは以下のサイトを参考にトラッキングを行ってみる  
https://ensekitt.hatenablog.com/entry/2017/12/21/200000

In [0]:
#driveとの連携
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd /content/drive/My Drive/School Project/System Design Project/recognize_num/

/content/drive/My Drive/School Project/System Design Project/recognize_num


In [0]:
import cv2
import numpy as np

In [0]:
#動画ファイルの読み込み
cap = cv2.VideoCapture('/content/drive/My Drive/School Project/System Design Project/recognize_num/movie/num.mp4')
print(cap.isOpened())#読み込めたかどうか

True


In [0]:
#とりあえず１フレーム読み込む
ret, frame = cap.read()

In [0]:
#文字部分の領域をrect_listに書き込み
f_frame = frame.astype(np.uint8)#int変換
bulr_img =cv2.GaussianBlur(f_frame,(15,15), 0)#平滑化
gray_img = cv2.cvtColor(bulr_img, cv2.COLOR_RGB2GRAY)#グレースケール
threshold_value=100 #二値化の閾値
ret, thresh_img = cv2.threshold(gray_img, threshold_value, 255, cv2.THRESH_BINARY)#二値化
fc_img,contours, hierarchy = cv2.findContours(thresh_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)#矩形領域の切り取り
rect_list = [0,0,0,0]
for i in range(0, len(contours)):
    if len(contours[i]) > 0:
        if hierarchy[0][i][3] != 0:
            continue
        rect=( contours[i])
        rect_list = np.vstack([rect_list,np.array(cv2.boundingRect(rect))])
rect_list= np.delete(rect_list, 0, 0)
rect_list

array([[ 89, 330,  77, 117],
       [439, 325,  45,  89],
       [292, 321,  59,  97],
       [367, 181,  60,  95],
       [505, 170,  47,  87],
       [193, 167,  75, 102],
       [ 56,  21,  35,  87],
       [246,  17,  79,  84],
       [426,   7,  49,  83]])

In [0]:
#trackking手法の選択
tracker = cv2.TrackerKCF_create()
ok = tracker.init(frame, tuple(rect_list[3]))

In [0]:
from pathlib import Path
output_dir = Path("output_track")
output_dir.mkdir(exist_ok=True)
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # フレーム数を取得する。
n_digits = len(str(n_frames))  

In [0]:
#トラッキング
while True: 
    ret, frame = cap.read()#1フレーム読み込み
    if ret == False:#フレームが読み込めなかったら出る
        break
    track, bbox = tracker.update(frame)
    i_bbox = np.array(bbox,dtype=np.int32)
    cv2.rectangle(frame, (i_bbox[0], i_bbox[1]), (i_bbox[0] + i_bbox[2], i_bbox[1] + i_bbox[3]), (0, 100, 0), 2)
    frame_no = int(cap.get(cv2.CAP_PROP_POS_FRAMES))  #フレーム番号
    save_path = output_dir / f"frame_{frame_no:0{n_digits}d}.png"#パス
    cv2.imwrite(str(save_path), frame)#セーブ
cap.release()  

In [0]:
#画像から動画作成
img_paths = Path("output_track") / "frame_%03d.png"

# VideoCapture を作成する。
rec_video = cv2.VideoCapture(str(img_paths))
width = int(rec_video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(rec_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = 15
print(f"width: {width}, height: {height}, fps: {fps}")

# VideoWriter を作成する。
fourcc = cv2.VideoWriter_fourcc(*"DIVX")
writer = cv2.VideoWriter("output_track.avi", fourcc, fps, (width, height))

while True:
    # 1フレームずつ取得する。
    ret, frame = rec_video.read()
    if not ret:
        break  # フレームの取得に失敗または動画の末尾に到達した場合

    writer.write(frame)  # フレームを書き込む。

writer.release()
rec_video.release()

#結果
ええ感じ！  
複数trackingしたかったらtrackerを増やせば行けるはず。